In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data.csv')

In [6]:
df.shape

(24691, 58)

In [7]:
df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,STL,BLK,TOV,PF,PTS,TrueSalary,Yrs Experience,Height,Weight,Rounded Position
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,NaN,NaN,NaN,217.0,458.0,0,0.0,0.0,0.0,0.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,NaN,NaN,NaN,99.0,279.0,0,0.0,0.0,0.0,0.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,NaN,NaN,NaN,192.0,438.0,0,0.0,0.0,0.0,0.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,29.0,63.0,0,0.0,0.0,0.0,0.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,NaN,NaN,NaN,27.0,59.0,0,0.0,0.0,0.0,0.0


In [8]:
df.tail()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,STL,BLK,TOV,PF,PTS,TrueSalary,Yrs Experience,Height,Weight,Rounded Position
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,...,62.0,58.0,65.0,189.0,639.0,0,0.0,0.0,0.0,0.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,...,7.0,21.0,20.0,61.0,178.0,0,0.0,0.0,0.0,0.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,...,2.0,5.0,3.0,17.0,23.0,0,0.0,0.0,0.0,0.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,...,15.0,16.0,40.0,78.0,240.0,0,0.0,0.0,0.0,0.0
24690,24690,2017.0,Ivica Zubac,C,19.0,LAL,38.0,11.0,609.0,17.0,...,14.0,33.0,30.0,66.0,284.0,0,0.0,0.0,0.0,0.0
